# Processo de Representação e Indexação


## 1. Index Invetido Simples

### Preparação dos Documentos

#### Visão lógica

In [ ]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Coleta
corpus = {
    'doc1':"To do is to be. To be is to do!",
    'doc2':"To be or not to be. I am what I am.",
    'doc3':"I think therefore I am. Do be do be do.",
    'doc4':"Do do do, da da da Let it be, let it be.",
}

#Visão lógica
doc_palavras = {} 

for key, doc in corpus.items():
  #Tokenização
  tokens = nltk.word_tokenize(doc)
  #Normalização
  doc_palavras[key] = [p.lower() for p in tokens if p.isalpha()]

doc_palavras

{'doc1': ['to', 'do', 'is', 'to', 'be', 'to', 'be', 'is', 'to', 'do'],
 'doc2': ['to', 'be', 'or', 'not', 'to', 'be', 'i', 'am', 'what', 'i', 'am'],
 'doc3': ['i', 'think', 'therefore', 'i', 'am', 'do', 'be', 'do', 'be', 'do'],
 'doc4': ['do',
  'do',
  'do',
  'da',
  'da',
  'da',
  'let',
  'it',
  'be',
  'let',
  'it',
  'be']}

#### Representação Termo x Documento

Representação é útil:

*   Modelos de RI
*   Classificação de texto
*   Análise de texto

In [ ]:
#Representação TfIdf utilizando TFIdf do Sklean
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=None)

docs = []
for _, doc in corpus.items():
  docs.append(doc)
  
#Criar matriz TFIDF dos documentos
X = vectorizer.fit_transform(docs)

# To data frame
df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names(),index = ['Doc%s'%i for i in range(1,X.shape[0]+1)])
df

,am,be,da,do,is,it,let,not,or,therefore,think,to,what
Doc1,0.000000,0.255666,0.000000,0.312717,0.489931,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.772535,0.000000
Doc2,0.523806,0.346702,0.000000,0.000000,0.000000,0.000000,0.000000,0.332191,0.332191,0.000000,0.000000,0.523806,0.332191
Doc3,0.290266,0.384249,0.000000,0.704987,0.000000,0.000000,0.000000,0.000000,0.000000,0.368166,0.368166,0.000000,0.000000
Doc4,0.000000,0.223758,0.643179,0.410533,0.000000,0.428786,0.428786,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Indexação simples

O `hashedindex` fornece uma estrutura de índice invertida simples de usar que é flexível o suficiente para trabalhar com todos os tipos de casos de uso.

In [ ]:
!pip install hashedindex -q

In [ ]:
# Processo de Indexação
import hashedindex

In [ ]:
# Criação do Índice Invertido
index = hashedindex.HashedIndex()

for doc_name , terms in doc_palavras.items():
  for t in terms:
    #Adiciona Termo e Documento
    index.add_term_occurrence(t, doc_name)

index.to_dict()

{'documents': Counter({'doc1': 10, 'doc2': 11, 'doc3': 10, 'doc4': 12}),
 'terms': {'am': Counter({'doc2': 2, 'doc3': 1}),
  'be': Counter({'doc1': 2, 'doc2': 2, 'doc3': 2, 'doc4': 2}),
  'da': Counter({'doc4': 3}),
  'do': Counter({'doc1': 2, 'doc3': 3, 'doc4': 3}),
  'i': Counter({'doc2': 2, 'doc3': 2}),
  'is': Counter({'doc1': 2}),
  'it': Counter({'doc4': 2}),
  'let': Counter({'doc4': 2}),
  'not': Counter({'doc2': 1}),
  'or': Counter({'doc2': 1}),
  'therefore': Counter({'doc3': 1}),
  'think': Counter({'doc3': 1}),
  'to': Counter({'doc1': 4, 'doc2': 2}),
  'what': Counter({'doc2': 1})}}

## 2. Indexação com ElasticSearch

### Servidor ES

In [ ]:
# Instalar servidor ES

#Instalar Java
!apt install default-jdk > /dev/null

#Donwload ES
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.8.0-linux-x86_64.tar.gz -q --show-progress

#Descompacta
!tar -xzf elasticsearch-7.8.0-linux-x86_64.tar.gz

#Atribui permissão ao usuario "daemon"
!chown -R daemon:daemon elasticsearch-7.8.0

# Iniciar o servidor
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.8.0/bin/elasticsearch'],stdout=PIPE, stderr=STDOUT,preexec_fn=lambda: os.setuid(1))  
# as daemon)

# Instalar module python
!pip install elasticsearch -q
!pip install elasticsearch-dsl -q



elasticsearch-7.8.0 100%[===================>] 304.33M  88.4MB/s    in 3.4s    
     |████████████████████████████████| 194kB 3.5MB/s 
     |████████████████████████████████| 61kB 2.1MB/s 


In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch()
if es.ping():
  print("ElasticSearch ONLINE !")

ElasticSearch ONLINE !


### 2.1 Indexação no ES



1.   Manualmente utilizando URL da Api
2.   módulo `elasticsearch`
2.   módulo `elasticsearch-dsl`



https://www.elastic.co/guide/en/elasticsearch/reference/master/indices-create-index.html

https://elasticsearch-py.readthedocs.io/en/master/api.html#indices

### 2.2 Criar Índice

#### Domínio do Índice

Proposições Parlamentares https://dadosabertos.camara.leg.br/swagger/api.html#staticfile

Proposições por ano de apresentação

Arquivos em que cada registro contém dados sobre uma proposição apresentada à Câmara dos Deputados para deliberação, como identificador universal (URI), sigla, número, ano, ementa, temas e keywords, informações sobre a tramitação mais recente, proposições a que se relacionam, etc.

Caminho para download: http://dadosabertos.camara.leg.br/arquivos/proposicoes/{formato}/proposicoes-{ano}.{formato}, em que:

{ano} é o ano de apresentação das proposições
{formato} pode ser “csv”, “xlsx”, "ods", “json” ou “xml”.

In [ ]:
#baixar dados
!wget https://dadosabertos.camara.leg.br/arquivos/proposicoes/csv/proposicoes-2020.csv -O proposicoes-2020.csv -q 

In [ ]:
# Ler documentos com Pandas
import pandas as pd

proposicoes = pd.read_csv("proposicoes-2020.csv", sep=";", encoding='utf-8', error_bad_lines=False)
proposicoes.head()

,id,uri,siglaTipo,numero,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords,dataApresentacao,uriOrgaoNumerador,uriPropAnterior,uriPropPrincipal,uriPropPosterior,urlInteiroTeor,urnFinal,ultimoStatus_dataHora,ultimoStatus_sequencia,ultimoStatus_uriRelator,ultimoStatus_idOrgao,ultimoStatus_siglaOrgao,ultimoStatus_uriOrgao,ultimoStatus_regime,ultimoStatus_descricaoTramitacao,ultimoStatus_idTipoTramitacao,ultimoStatus_descricaoSituacao,ultimoStatus_idSituacao,ultimoStatus_despacho,ultimoStatus_url
0,2077498,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,1600,2020,139,Projeto de Lei,"Altera a Lei nº 8.069, de 13 de julho de 1990,...",NaN,"Alteração, Estatuto da Criança e do Adolescent...",2016-02-18T12:56:57,https://dadosabertos.camara.leg.br/api/v2/orga...,NaN,NaN,http://legis.senado.leg.br/dadosabertos/materi...,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,2020-04-06T18:44:00,60,https://dadosabertos.camara.leg.br/api/v2/depu...,100229.0,SEPRO,https://dadosabertos.camara.leg.br/api/v2/orga...,"Ordinária (Art. 151, III, RICD)",Apresentação de Proposição,100,Aguardando Recebimento,917.0,Apresentação da Emenda/Substitutivo do Senado ...,http://www.camara.gov.br/proposicoesWeb/prop_m...
1,2116186,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,1599,2020,139,Projeto de Lei,Institui a Semana Nacional de Prevenção e Comb...,NaN,"Criação, Semana Nacional de Prevenção e Combat...",2016-11-07T18:01:00,https://dadosabertos.camara.leg.br/api/v2/orga...,NaN,NaN,http://legis.senado.leg.br/dadosabertos/materi...,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,2020-04-06T18:17:00,64,https://dadosabertos.camara.leg.br/api/v2/depu...,100229.0,SEPRO,https://dadosabertos.camara.leg.br/api/v2/orga...,"Ordinária (Art. 151, III, RICD)",Apresentação de Proposição,100,Aguardando Recebimento,917.0,Apresentação da Emenda/Substitutivo do Senado ...,http://www.camara.gov.br/proposicoesWeb/prop_m...
2,2236217,https://dadosabertos.camara.leg.br/api/v2/prop...,SUG,1,2020,285,Sugestão,Sugere estudo visando verificar a atuação do M...,NaN,NaN,2020-01-03T11:09:00,https://dadosabertos.camara.leg.br/api/v2/orga...,NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,2020-01-03T11:09:00,1,NaN,5438.0,CLP,https://dadosabertos.camara.leg.br/api/v2/orga...,"Ordinária (Art. 151, III, RICD)",Apresentação de Proposição,100,Aguardando Designação de Relator,907.0,"Apresentação da Sugestão n. 1/2020, pelo Socie...",http://www.camara.gov.br/proposicoesWeb/prop_m...
3,2236218,https://dadosabertos.camara.leg.br/api/v2/prop...,SUG,2,2020,285,Sugestão,Sugere audiência pública para debater assuntos...,NaN,NaN,2020-01-03T11:24:00,https://dadosabertos.camara.leg.br/api/v2/orga...,NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,2020-01-03T11:24:00,1,NaN,5438.0,CLP,https://dadosabertos.camara.leg.br/api/v2/orga...,"Ordinária (Art. 151, III, RICD)",Apresentação de Proposição,100,Aguardando Designação de Relator,907.0,"Apresentação da Sugestão n. 2/2020, pela Socie...",http://www.camara.gov.br/proposicoesWeb/prop_m...
4,2236222,https://dadosabertos.camara.leg.br/api/v2/prop...,RIC,1,2020,148,Requerimento de Informação,Requer informações detalhadas ao Ministro da C...,NaN,NaN,2020-01-03T14:50:13,https://dadosabertos.camara.leg.br/api/v2/orga...,NaN,NaN,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,2020-03-18T00:00:00,22,https://dadosabertos.camara.leg.br/api/v2/depu...,5470.0,1SECM,https://dadosabertos.camara.leg.br/api/v2/orga...,NaN,Providência Interna - Comunicação de resposta ...,1056,Aguardando Remessa ao Arquivo,931.0,Encaminhamento de resposta por meio do sistema...,NaN


#### Criar

In [ ]:
# Utilizando Elasticsearch-dsl
from elasticsearch_dsl.connections import  connections
from elasticsearch_dsl import  Index

connections.create_connection()

index_name = 'politica-index'
politica = Index(index_name)

politica.settings(
    number_of_shards=2
)

if not politica.exists():
  politica.create()
else:
  politica.delete()

politica.create()

{'acknowledged': True, 'index': 'politica-index', 'shards_acknowledged': True}

In [ ]:
politica.get()

{'politica-index': {'aliases': {},
  'mappings': {},
  'settings': {'index': {'creation_date': '1594478251720',
    'number_of_replicas': '1',
    'number_of_shards': '2',
    'provided_name': 'politica-index',
    'uuid': 'CEc8vLVwSLK02btogMjyGw',
    'version': {'created': '7080099'}}}}}

### 2.3 Análise dos documentos

In [ ]:
# ANALISADOR
from elasticsearch_dsl.analysis import analyzer, char_filter, tokenizer, token_filter

#Char
char_filter_ptbr = char_filter('html_strip')

#Tokenizador
tokenizador_ptbr = tokenizer("standard")

#Analisador
stop_ptbr = token_filter("brazilian_stop", type="stop", stopwords="_brazilian_")
stem_ptbr = token_filter("brazilian_stemmer", type="stemmer", stopwords="_brazilian_")


pt_analyzer = analyzer(
    'pt_analyzer',
    tokenizer= tokenizador_ptbr,
    filter = [stop_ptbr, stem_ptbr, "lowercase"],
    char_filter = [char_filter_ptbr]
)


In [ ]:
res = pt_analyzer.simulate("<br> teste Aula <b> DRI !")
[t.token for t in res.tokens]

['test', 'aula', 'dri']

### 2.4 Mapeamento



https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping.html

In [ ]:
# Criar a entidade
from elasticsearch_dsl import Document, Text, Date, Keyword

class Projeto(Document):
  ementa = Text(analyzer = pt_analyzer)
  topicos = Text()
  data = Date() 
  tipo = Keyword()


In [ ]:
# Mapear no index
politica.document(Projeto)
politica.close()
Projeto.init()
politica.open()



{'acknowledged': True, 'shards_acknowledged': True}

In [ ]:
#Visualizar Mappeamento
politica.get_mapping()

{'politica-index': {'mappings': {'properties': {'data': {'type': 'date'},
    'ementa': {'analyzer': 'pt_analyzer', 'type': 'text'},
    'tipo': {'type': 'keyword'},
    'topicos': {'type': 'text'}}}}}

## 3 Manipular documentos

Ciclo de vida do documento (CRUD)


1.   Criar
2.   Ler 
3.   Atualizar
4.   Deletar


#### 3.1 Criar Documento

In [ ]:
# Documento de Exemplo

projeto1 = Projeto()

projeto1.meta.id = 1
projeto1.tipo="PL",
projeto1.ementa='Altera a Lei nº 8.069, de 13 de julho de 1990, que "dispõe sobre o Estatuto da Criança e do Adolescente, e dá outras providências", para proibir a venda de produtos fumígenos a crianças e adolescentes.',
projeto1.data="2020-01-03T11:24:00",
projeto1.topicos="Alteração, Estatuto da Criança e do Adolescente, proibição, comercialização, criança, adolescente, produto fumígeno, tabaco, cachimbo, narguilé, cigarro."

projeto1.to_dict()

{'data': ['2020-01-03T11:24:00'],
 'ementa': ('Altera a Lei nº 8.069, de 13 de julho de 1990, que "dispõe sobre o Estatuto da Criança e do Adolescente, e dá outras providências", para proibir a venda de produtos fumígenos a crianças e adolescentes.',),
 'tipo': ('PL',),
 'topicos': 'Alteração, Estatuto da Criança e do Adolescente, proibição, comercialização, criança, adolescente, produto fumígeno, tabaco, cachimbo, narguilé, cigarro.'}

In [ ]:
#Adiconar documento
projeto1.save()

'created'

#### 3.2 Ler Documento

In [ ]:
#ler documento
projeto2 =  Projeto.get(id=1)
projeto2.to_dict()

NotFoundError: ignored

#### 3.3 Atualizar Documento

In [ ]:
#Atualizar
projeto2 =  Projeto.get(id=1)
projeto2.tipo = "PL2"
projeto2.save()

'updated'

#### 3.4 Remover Documento

In [ ]:
#Deletar documento
projeto2 =  Projeto.get(id=1)
projeto2.delete()

#### 3.5 Carregar em lote

In [ ]:
#Usando Bulk
from elasticsearch.helpers import bulk

projeto1 = Projeto()

projeto1.meta.id = 1
projeto1.tipo="PL",
projeto1.ementa='Altera a Lei nº 8.069, de 13 de julho de 1990, que "dispõe sobre o Estatuto da Criança e do Adolescente, e dá outras providências", para proibir a venda de produtos fumígenos a crianças e adolescentes.',
projeto1.data="2020-01-03T11:24:00",
projeto1.topicos="Alteração, Estatuto da Criança e do Adolescente, proibição, comercialização, criança, adolescente, produto fumígeno, tabaco, cachimbo, narguilé, cigarro."


projeto2 = Projeto()

projeto2.meta.id = 2
projeto2.tipo="PL2",
projeto2.ementa='Altera a Lei nº 8.069, de 13 de julho de 1990, que "dispõe sobre o Estatuto da Criança e do Adolescente, e dá outras providências", para proibir a venda de produtos fumígenos a crianças e adolescentes.',
projeto2.data="2020-01-03T11:24:00",
projeto2.topicos="Alteração, Estatuto da Criança e do Adolescente, proibição, comercialização, criança, adolescente, produto fumígeno, tabaco, cachimbo, narguilé, cigarro."


projetos = [projeto1, projeto2]

bulk(client=politica.connection, actions=(p.to_dict(include_meta=True) for p in projetos),index=politica._name)


(2, [])

In [ ]:
from elasticsearch.helpers import bulk

proposicoes = proposicoes.fillna('')

projetos = []
for p in proposicoes.itertuples(index=False):
  projeto1 = Projeto()

  projeto1.meta.id = p.id
  projeto1.tipo = p.siglaTipo
  projeto1.ementa = p.ementa
  projeto1.data= p.dataApresentacao
  projeto1.topicos=p.keywords

  projetos.append(projeto1)


bulk(client=politica.connection, actions=(p.to_dict(include_meta=True) for p in projetos),index=politica._name)


(20024, [])

{'politica-index': {'aliases': {},
  'mappings': {'properties': {'data': {'type': 'date'},
    'ementa': {'analyzer': 'pt_analyzer', 'type': 'text'},
    'tipo': {'type': 'keyword'},
    'topicos': {'type': 'text'}}},
  'settings': {'index': {'analysis': {'analyzer': {'pt_analyzer': {'char_filter': ['html_strip'],
       'filter': ['brazilian_stop', 'brazilian_stemmer', 'lowercase'],
       'tokenizer': 'standard',
       'type': 'custom'}},
     'filter': {'brazilian_stemmer': {'stopwords': '_brazilian_',
       'type': 'stemmer'},
      'brazilian_stop': {'stopwords': '_brazilian_', 'type': 'stop'}}},
    'creation_date': '1594478251720',
    'number_of_replicas': '1',
    'number_of_shards': '2',
    'provided_name': 'politica-index',
    'uuid': 'CEc8vLVwSLK02btogMjyGw',
    'version': {'created': '7080099'}}}}}

## Lab 2 Prático

Atividade 2: indexar documentos no Elasticsearch. Utilize as atividades de exemplo acima como referência:


1.   Criar índice
2.   Análise dos documentos
3.   Mapeamento
4.   Inserir documentos no ES




### ATIVIDADE 2

Utilizando os arquivos baixados no Lab 1 para indexá-los no Elasticsearch

In [ ]:
#IMPORTS LIBS

In [ ]:
#COLETA

In [ ]:
#INDEXACAO

In [ ]:
#ANALISE

In [ ]:
#MAPEAMENTO

In [ ]:
#BULK